<a href="https://colab.research.google.com/github/Mohammdamin-Sedaghat/stockML/blob/Aviral/AAPL_data_gathering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libraries

Install pandas as it will be needed to store data in a dataframe and manipulate csv files.

In [4]:
!pip install pandas

# Importing libraries and data collection

Import important librarires such as *pandas*, *requests*, *io*, and *datetime*.

If working with files on Google Drive make sure to import the drive *files* and *drive* packages from *google.colab*.

Replace *api_key* with an alpha vantage API key. The key can be obtained at this [website](https://www.alphavantage.co/support/#api-key).

```
api_key = 'your api key'  # insert alpha vantage API key here
```
Change the *symbol* variable with the ticker symbol if you would like data for a different stock. AAPL was used for this project.

Make sure to alter the *end_date* and *start_date* variables to time period you would like data for. The reason for the start date being Jan 1 2000 will be explained a little further.

The while loop runs and gets a response (response status code 200) from the URL as long as the *current_date* is after the *start_date* and while the counter, *count*, is less than 25. This counter was necessary as the free API only allows for 25 requests per day. For paid versions, the counter can be altered or removed entirely. The January 1 2000 starting date was entered as a means of reusing the same piece of code every day for data collection without having to change too many parameters.

The csv data returned from the URL is formatted in a *monthly_df* dataframe with the respective columns and each dataframe is appended to the *all_data* list.

The dataframes are then combined into a single dataframe at the end called *AAPL_data*. This dataframe is then stored as a csv file to drive and local machine.






In [12]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from google.colab import files
from google.colab import drive

drive.mount('/content/drive')

api_key = 'your API key'
symbol = 'AAPL'

# starting and ending dates
end_date = datetime(2024, 10, 31)
start_date = datetime(2000, 1, 1)

all_data = []

current_date = end_date
count = 0
while current_date >= start_date and count < 25:
    try:
        month = current_date.strftime('%Y-%m')
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}&datatype=csv&month={month}&outputsize=full"


        response = requests.get(url)


        if response.status_code == 200:
            monthly_df = pd.read_csv(StringIO(response.text))
            monthly_df = monthly_df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]  # Include columns


            # append monthly to all data and print confirmation
            all_data.append(monthly_df)
            print(f'Data added for {month}')

        else:
            print(f'Data for {month} NOT added. TRY AGAIN. Status code: {response.status_code}')

    except requests.exceptions.RequestException as e:
        print(f'Request failed for {month}: {e}' )
        break

    current_date -= relativedelta(months=1)  # Subtract exactly one month
    count += 1

#combine all df
AAPL_data = pd.concat(all_data, ignore_index = True)

print(AAPL_data.head(2)) #check the structure of the df

#save to csv
AAPL_data.to_csv("AAPL_data.csv", index=False)
files.download("AAPL_data.csv") #save csv to local machine
AAPL_data.to_csv("/content/drive/My Drive/AAPL_data.csv", index=False)

print(f'Data extracted from {current_date.strftime("%Y-%m")} to {end_date.strftime("%Y-%m")}. CSV saved as "AAPL_data.csv"')
print(f'Data extracted starting from {current_date} to {end_date}. Next time start data collection from {current_date} and move backwards')






Mounted at /content/drive
Data added for 2024-10
Data added for 2024-10
Data added for 2024-09
Data added for 2024-08
Data added for 2024-07
Data added for 2024-06
Data added for 2024-05
Data added for 2024-04
Data added for 2024-03
Data added for 2024-02
Data added for 2024-01
Data added for 2023-12
Data added for 2023-11
Data added for 2023-10
Data added for 2023-09
Data added for 2023-08
Data added for 2023-07
Data added for 2023-06
Data added for 2023-05
Data added for 2023-04
Data added for 2023-03
Data added for 2023-02
Data added for 2023-01
Data added for 2022-12
Data added for 2022-11
             timestamp      open      high       low    close  volume
0  2024-10-31 19:59:00  221.4699  221.6359  221.3662  221.501    8547
1  2024-10-31 19:58:00  221.5759  221.6059  221.4573  221.536    8423


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data extracted from 2022-10 to 2024-10. CSV saved as "AAPL_data.csv"
Data extracted starting from 2022-10-12 00:00:00 to 2024-10-31 00:00:00. Next time start data collection from 2022-10-12 00:00:00 and move backwards


Code tht checks general shape of the *AAPL_data* dataframe. The first run-through had duplicates whicj are removed by this code.

In [17]:
AAPL_data.shape

#remove repeated data for 2024-10
AAPL_data_new = AAPL_data.drop_duplicates(subset=['timestamp'], keep='first')
AAPL_data_new.shape

# test print
# AAPL_data_new.head(2)

AAPL_data = AAPL_data_new
AAPL_data.shape

AAPL_data.head(2)

,timestamp,open,high,low,close,volume
0,2024-10-31 19:59:00,221.4699,221.6359,221.3662,221.501,8547
1,2024-10-31 19:58:00,221.5759,221.6059,221.4573,221.536,8423


Checking size post duplicate removal.

In [18]:
AAPL_data.shape

(455709, 6)

Describing data collected.

In [19]:
AAPL_data.describe()

,open,high,low,close,volume
count,455709.000000,455709.000000,455709.000000,455709.000000,4.557090e+05
mean,182.469192,182.538899,182.387205,182.469234,7.034138e+04
std,26.435696,26.448589,26.418197,26.435999,7.217024e+05
min,122.911100,122.960600,122.881400,122.911100,1.000000e+00
25%,168.775300,168.829000,168.690300,168.775300,4.650000e+02
50%,181.108900,181.172400,181.033600,181.108900,4.871000e+03
75%,193.360400,193.439800,193.293600,193.360400,7.689500e+04
max,247.385800,249.740300,236.959000,246.867000,2.431802e+08


Similar to original code but also reads existing csv, converts the data into a dataframe called *AAPL_data_existing* and combines this data with more data collected before saving that as a csv file.

In [8]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from google.colab import files
from google.colab import drive

api_key = 'your api key'
symbol = 'AAPL'

# starting and ending dates
end_date = datetime(2022, 10, 31)
start_date = datetime(2000, 1, 1)

all_data = []

current_date = end_date
count = 0
while current_date >= start_date and count < 25:
    try:
        month = current_date.strftime('%Y-%m')
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}&datatype=csv&month={month}&outputsize=full"


        response = requests.get(url)


        if response.status_code == 200:
            monthly_df = pd.read_csv(StringIO(response.text))
            monthly_df = monthly_df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]  # Include columns


            # append monthly to all data and print confirmation
            all_data.append(monthly_df)
            print(f'Data added for {month}')

        else:
            print(f'Data for {month} NOT added. TRY AGAIN. Status code: {response.status_code}')

    except requests.exceptions.RequestException as e:
        print(f'Request failed for {month}: {e}' )
        break

    current_date -= relativedelta(months=1)  # Subtract exactly one month
    count += 1

#combine all df
AAPL_data = pd.concat(all_data, ignore_index = True)

print(AAPL_data.head(2)) #check the structure of the df

AAPL_data_existing = pd.read_csv("/content/drive/My Drive/AAPL_data.csv")

AAPL_data = pd.concat([AAPL_data_existing, AAPL_data], ignore_index=True)

#save to csv
AAPL_data.to_csv("AAPL_data.csv", index=False)
files.download("AAPL_data.csv") #save csv to local machine
AAPL_data.to_csv("/content/drive/My Drive/AAPL_data.csv", index=False)

print(f'Data extracted from {current_date.strftime("%Y-%m")} to {end_date.strftime("%Y-%m")}. CSV saved as "AAPL_data.csv"')
print(f'Data extracted starting from {current_date} to {end_date}. Next time start data collection from {current_date} and move backwards')






Data added for 2022-10
Data added for 2022-09
Data added for 2022-08
Data added for 2022-07
Data added for 2022-06
Data added for 2022-05
Data added for 2022-04
Data added for 2022-03
Data added for 2022-02
Data added for 2022-01
Data added for 2021-12
Data added for 2021-11
Data added for 2021-10
Data added for 2021-09
Data added for 2021-08
Data added for 2021-07
Data added for 2021-06
Data added for 2021-05
Data added for 2021-04
Data added for 2021-03
Data added for 2021-02
Data added for 2021-01
Data added for 2020-12
Data added for 2020-11
Data added for 2020-10
             timestamp      open      high       low     close  volume
0  2022-10-31 19:59:00  151.6155  151.6550  151.6155  151.6155     762
1  2022-10-31 19:58:00  151.6253  151.6253  151.6056  151.6155     600


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data extracted from 2020-09 to 2022-10. CSV saved as "AAPL_data.csv"
Data extracted starting from 2020-09-28 00:00:00 to 2022-10-31 00:00:00. Next time start data collection from 2020-09-28 00:00:00 and move backwards
